From https://nbviewer.org/github/AllenDowney/AstronomicalData/blob/main/02_coords.ipynb

In [5]:
# a demo of an ASQL cone search
import astropy.units as u
from astroquery.gaia import Gaia


query_cone = """SELECT 
COUNT(source_id)
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT(ra, dec),
  CIRCLE({ra}, {dec}, {radius})
)
"""


ra=88.8
dec=7.4
radius=0.833333
query=query_cone.format(ra=ra,dec=dec,radius=radius)

job = Gaia.launch_job(query)
results = job.get_results()
results

COUNT
int64
63173


In [18]:
from astropy.coordinates import SkyCoord

# create a query
 
# the coordinates of the corners of our search polygon
# this list of points is generated with Gala library which is not installing for me rn
# so I generated it in a Collab notebook and am hardcoding
# 
point_list="146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619"

columns = 'source_id, ra, dec, pmra, pmdec, parallax'

query_base = """ SELECT
    {columns}
    FROM gaiadr2.gaia_source
    WHERE 
        parallax < 1
        AND bp_rp BETWEEN -0.75 AND 2
        AND 1 = CONTAINS(
            POINT(ra,dec),
            POLYGON({point_list})
        )
"""

query = query_base.format(columns=columns, point_list=point_list)
print(query) 


 SELECT
    source_id, ra, dec, pmra, pmdec, parallax
    FROM gaiadr2.gaia_source
    WHERE 
        parallax < 1
        AND bp_rp BETWEEN -0.75 AND 2
        AND 1 = CONTAINS(
            POINT(ra,dec),
            POLYGON(146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619)
        )



In [ ]:
# run the query
job = Gaia.launch_job_async(query)
results = job.get_results()
len(results)

In [ ]:
# and save to a FITS file
filename = "gd1_data.fits"
results.write(filename, overwrite=True)

from os.path import getsize
MB = 1024*1024
getsize(filename)/MB

6.4324951171875